In [1]:
# Install required python packages
!pip3 install --upgrade --user google-cloud-aiplatform

In [2]:
# restart the kernet after libraries are loaded
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Define project information
PROJECT_ID = "qwiklabs-gcp-02-be6eced45518"  # @param {type:"string"}
LOCATION = "us-east1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
# Task 2
# import software libraries required
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [4]:
# Task 2.1
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
model = GenerativeModel("gemini-1.0-pro-001")

In [14]:
# Task 2.2
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
function_name = "get_current_weather"
get_current_weather_func = FunctionDeclaration(
    name=function_name,
    description="Get the current weather in a given location",
    # Function parameters are specified in OpenAPI JSON schema format
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)


In [15]:
# Task 2.3
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

In [17]:
# Task 2.4
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
# Define the user's prompt in a Content object that we can reuse in model calls
user_prompt_content = Content(
    role="user",
    parts=[
        Part.from_text("What is the weather like in Boston?"),
    ],
)

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)

function_call = response.candidates[0].function_calls[0]

# Check the function name that the model responded with, and make an API call to an external system
if function_call.name == function_name:
    # Extract the arguments to use in your API call
    location = function_call.args["location"]  # noqa: F841

    # Here you can use your preferred method to make an API request to fetch the current weather, for example:
    # api_response = requests.post(weather_api_url, data={"location": location})

    # In this example, we'll use synthetic data to simulate a response payload from an external API
    api_response = """{ "location": "Boston, MA", "temperature": 38, "description": "Partly Cloudy",
                    "icon": "partly-cloudy", "humidity": 65, "wind": { "speed": 10, "direction": "NW" } }"""

# Return the API response to Gemini so it can generate a model response or request another function call
response = model.generate_content(
    [
        user_prompt_content,  # User prompt
        response.candidates[0].content,  # Function call response
        Content(
            parts=[
                Part.from_function_response(
                    name=function_name,
                    response={
                        "content": api_response,  # Return the API response to Gemini
                    },
                ),
            ],
        ),
    ],
    tools=[weather_tool],
)

# Get the model response
print(response.text)

The weather in Boston is partly cloudy with a temperature of 38 degrees Fahrenheit. The humidity is 65% and the wind is blowing at 10mph from the northwest.


In [20]:
# Task 3
# Run the following cell to import required libraries 
from vertexai.generative_models import (
    GenerationConfig,
    Image,
    Part,
)

In [32]:
# Task 3.1
# Load the correct Gemini model use the following documentation to assist:
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview#supported-use-cases
multimodal_model = GenerativeModel("gemini-1.5-flash-001")

In [33]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

In [47]:
# Task 3.2 Generate a video description
# In this cell, update the prompt to ask Gemini to describe the video URL referenced.
# You can use the documentation at the following link to assist.
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/sdk-for-gemini/gemini-sdk-overview-reference#generate-content-from-video
# 
# Video URI: gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4
# 
prompt = """
What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?
"""
video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4",
    mime_type="video/mp4",
)
contents = [video, prompt]

response = multimodal_model.generate_content(contents)

#print("-------Prompt--------")
#print_multimodal_prompt(contents)
print(response.text)

This video shows the beautiful harbor in Antalya, Turkey.  You can see the cliffs, the harbor, and the calm waters. Antalya is located on the Mediterranean coast of Turkey.  It's a great place to visit for its history, beaches, and culture.

Here are 5 other places in the world that look similar to Antalya:

1.  **Dubrovnik, Croatia:** Dubrovnik is another beautiful coastal city with stunning cliffs, a historic harbor, and crystal-clear waters.

2. **Santorini, Greece:** Santorini is famous for its white-washed villages perched atop dramatic cliffs overlooking the Aegean Sea. 

3. **Positano, Italy:** Positano is a picturesque village on the Amalfi Coast, known for its colorful houses clinging to the steep cliffs above the Tyrrhenian Sea.

4. **Monemvasia, Greece:** This fortified town on a rocky peninsula in the Peloponnese region is another great example of a coastal city with stunning cliffside views.

5. **Faro, Portugal:** This charming city is situated at the mouth of the Ria For